### Testing SimOOS
This notebook contains tests of SimOOS algorithm on R6B dataset.

In [1]:
import os
os.chdir('../..')
os.getcwd()

'/Users/sbokupripeku/git/work/examples/costly_nonstationary_bandits'

In [2]:
%load_ext autoreload
%autoreload 2

import pickle
import matplotlib.pyplot as plt
import numpy as np
import itertools 
import time

import algorithms
import costs
import evaluation
import dataset
from plotting.average_ctr import get_average_ctr_plot

In [3]:
%%time
import gc

with open("dataset/r6b/subsample/data_10.pickle", "rb") as f:
    gc.disable()
    data = pickle.load(f)
    gc.enable()

CPU times: user 3.18 s, sys: 1.03 s, total: 4.22 s
Wall time: 12.2 s


In [4]:
NUM_OF_TRIALS = 5000

In [5]:
p_random = algorithms.RandomPolicy()
ctr_random = evaluation.evaluate(p_random, data, stop_after=NUM_OF_TRIALS)

Random policy with ZeroCosts
Average reward: 0.0354
Execution time: 17.2s


In [6]:
all_contexts = np.stack(
 [
     ev.user_features for ev in data.events
 ]
)


In [7]:
N_FEATURES = 10

In [8]:
short_contexts = all_contexts[:, :N_FEATURES]

In [9]:
for event, short_x in zip(data.events, short_contexts+1):
    event.user_features = short_x

In [10]:
short_contexts.shape

(1751665, 10)

In [11]:
s = time.time()
p_simoos = algorithms.SimOOSAlgorithm(
    all_contexts=(short_contexts+1), 
    number_of_actions=data.n_arms,
    max_no_red_context=short_contexts.shape[1],
    beta_SimOOS=1,
    delta_SimOOS=0.1,
)
print(f"Took {time.time() - s} seconds")

Took 378.0268540382385 seconds


In [ ]:
ctr_simoos = evaluation.evaluate(
    p_simoos,
    data, 
    stop_after=NUM_OF_TRIALS,
    feature_costs=costs.ConstantCosts(5, [0.0000]*13)
)

In [ ]:
def plot_ctrs(ctr_dict, change_points=()):
    fig, ax = plt.subplots(1,1, figsize=(16, 8));
    
    N = 40
    
    max_vline=0
    
    for label, ctr in ctr_dict.items():
        ax.plot(ctr[N:], label=label)
        max_vline=max(max_vline, max(ctr[N:]))
    ax.set_xlabel('Trial')
    ax.set_ylabel('CTR')
    ax.set_title('Evaluation without costs')
    
    ax.vlines(change_points, 0, max_vline, label='change points', color='pink')

    plt.legend();
    plt.show();

In [ ]:
plot_ctrs({
    'Random': ctr_random,
    'SimOOS': ctr_simoos,
}
)


In [ ]:
vars(p_simoos)

In [ ]:
list(p_simoos.nu_t)

In [ ]:
list(p_simoos.upsilon_t)

In [ ]:
(p_simoos.upsilon_t == np.zeros(p_simoos.upsilon_t.shape)).flatten().all()

In [ ]:
list(p_simoos.N_t_o)

In [ ]:
list(p_simoos.selected_context_SimOOS)